#### 3. Explique como você faria a previsão do preço a partir dos dados. Quais
#### variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de
#### problema estamos resolvendo (regressão, classicação)? Qual modelo
#### melhor se aproxima dos dados e quais seus prós e contras? Qual medida de
#### performance do modelo foi escolhida e por quê?


### R= Para prever o preço de um imóvel com base nos dados, fiz uma abordagem estruturada de ciência de dados.
- Variaveis Utlizadas: latitude, longitude, room_type, numero_de_reviews, disponibilidade_365, calculado_host_listings_count
- Transformações Aplicadas: ( dropna()), room_type( OneHotEncoder), room_type( OneHotEncoder), ( StandardScaler), ( train_test_split).
### Motivos das Variáveis:
##### latitudeelongitude

- A localização é um dos principais fatores que influenciam o preço.
- Áreas centrais e turísticas geralmente têm preços mais altos.
 
##### room_type

- Convertido com One-Hot Encoding , pois é uma variável categórica.
- Quartos privados e inteiros tendem a ter preços mais altos do que quartos compartilhados.

##### numero_de_reviews

- Pode indicar popularidade do imóvel.
- Imóveis bem avaliados e com mais avaliações podem incluir preços mais altos.

##### disponibilidade_365

-Indica se o imóvel está disponível para locação o ano todo.
-Proprietários com alta disponibilidade podem ajustar preços conforme a demanda.

##### calculado_host_listings_count

-Pode indicar se o anfitrião é profissional ou amador.
-Hosts com muitos imóveis podem ajustar preços estrategicamente.

### Tipo de Problema: Regressão pois queremos prever um valor contínuo ( preço ).
### O MELHOR MODELO É: XGBoost
- PRÓS: Melhor desempenho, lida bem com dados tabulares
- CONTRAS: Mais difícil de curar

### Para avaliar o modelo, utilizamos duas métricas principais:

- MAE (Erro Médio Absoluto)
- RMSE (Raiz do Erro Quadrático Médio)
  
### Porque escolhi ela: 
- MAE fornece um erro médio fácil de interpretar.
- RMSE nos alerta sobre previsões muito erradas.
##### Juntas, elas ajudam a equilibrar precisão e robustez no modelo

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv(r"C:\Users\USER\Documents\teste_indicium_precificacao.csv")

features = ["latitude", "longitude", "room_type", "numero_de_reviews", "disponibilidade_365", "calculado_host_listings_count"]

df = df[features + ["price"]].dropna()  

ohe = OneHotEncoder(sparse_output=False, drop="first")
encoded_room_type = ohe.fit_transform(df[["room_type"]])
ohe_features = ohe.get_feature_names_out(["room_type"])
df_encoded = pd.DataFrame(encoded_room_type, columns=ohe_features)

df = df.drop(columns=["room_type"]).reset_index(drop=True)
df = pd.concat([df, df_encoded], axis=1)

X = df.drop(columns=["price"])
y = df["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")


MAE: 63.68
RMSE: 210.97
